# Homework 2 - balancing & binning (deadline 28. 11. 2021, 23:59)

In short, the main task is to play with balancing and binning to obtain the best results for the binary classification task.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can!

However, we **strongly recommend and require** the following:
* Follow the assignment step by step. Number each step.
* Properly comment all your steps. Comments are evaluated for 2 points of the total together with the final presentation of the solution. However, it is not desirable to write novels! 
* Do not leave the task to the last minute.
* Hand in a notebook that has already been run (i.e. do not delete outputs before handing in).

## What are you supposed to do:

  1. Download the dataset [here](https://www.dropbox.com/s/aq08ytozfplb97b/data.csv?dl=0).
  1. Use 2 binning methods (on features of your choice, with your choice of parameters) and comment on its effects on classification performance. _(3 points)_
  1. Use at least 2 other data balancing techniques of your choice on the dataset and comment the classification results. **Just copied code from tutorial 4 will not be accepted.** _(6 points)_
  1. Run all classification tests at least three times - once for unbalanced original data, twice for balanced data (try at least 2 balancing techniques), compare those results and discuss (give a comment). _(5 points)_
  
All your steps and following code **have to be commented!** Comments are evaluated for _2 points_ together with the final presentation of the solution.

**If you do all this properly, you will obtain 16 points.**

## Comments

  * Please follow the technical instructions from https://courses.fit.cvut.cz/NI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, she can (!) give you another chance to rework your homework and to obtain more points. However, this is not a given, so do your best! :)
  * English is not compulsory.
  
## Data description

* The dataset can be downloaded [here](https://www.dropbox.com/s/aq08ytozfplb97b/data.csv?dl=0).
* The data are devoted to the binary classification task, the aim is to predict the probability that a driver will initiate an auto insurance claim in the next year.
* Target feature is called 'y' and signifies whether or not a claim was filed for that policy holder.
* To fulfill the task one does not need to know the meaning of predictors.
* Predictors that belong to similar groupings are tagged as such in the feature names (e.g., ind, reg, car, calc). In addition, feature names include the postfix bin to indicate binary features and cat to indicate categorical features. Features without these designations are either continuous or ordinal. Values of -1 indicate that the feature was missing from the observation.
* While using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), control the shuffling of data by random_state parameter. Do not use shuffle=False, probably never (can cause systematic error).

In [2]:
# libraries
import numpy as np
import pandas as pd

from scipy.stats import norm, ttest_ind
from scipy.optimize import minimize_scalar

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import f_regression, mutual_info_regression, RFECV
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_log_error, make_scorer, mean_squared_error, confusion_matrix, accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification, make_blobs
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from imblearn.under_sampling import EditedNearestNeighbours, OneSidedSelection

ModuleNotFoundError: No module named 'imblearn'

In [45]:
# load data
df = pd.read_csv('data.csv')
df.head()

,y,xind_01,xind_02_cat,xind_03,xind_04_cat,xind_05_cat,xind_06_bin,xind_07_bin,xind_08_bin,xind_09_bin,...,xcalc_11,xcalc_12,xcalc_13,xcalc_14,xcalc_15_bin,xcalc_16_bin,xcalc_17_bin,xcalc_18_bin,xcalc_19_bin,xcalc_20_bin
0,0,2,1,5,1,6,0,1,0,0,...,4,0,2,8,0,0,0,1,1,0
1,0,2,1,6,0,0,0,0,0,1,...,9,2,3,7,0,0,1,0,0,0
2,0,2,1,9,0,0,1,0,0,0,...,5,0,2,6,0,1,0,1,1,0
3,0,1,1,6,0,0,1,0,0,0,...,3,0,4,8,0,0,1,0,0,0
4,0,1,1,5,1,0,0,1,0,0,...,4,3,1,2,0,1,0,0,1,0
5,0,1,1,2,1,0,0,1,0,0,...,9,2,3,5,0,0,1,0,0,0
6,0,2,1,3,0,0,0,0,0,1,...,8,4,1,6,1,1,0,0,0,0
7,0,2,1,3,1,0,0,1,0,0,...,8,1,3,7,0,1,1,0,1,0
8,0,0,2,1,0,0,1,0,0,0,...,5,3,2,6,0,1,0,0,0,0
9,0,5,1,2,1,0,0,1,0,0,...,6,0,3,8,0,1,1,0,1,0


In [32]:
# prepare data
y = df['y']
x = df.drop(['y'], axis=1)

### Binning

In [40]:
# Task 2.1 - binning 1 - equal width
x['xcalc_14_2'] = df['xcalc_14']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, random_state=42)
x_train['xcalc_14_2'] = pd.cut(x_train['xcalc_14'], 4).cat.codes

In [41]:
# classification:
x_train.drop(['xcalc_14'], axis = 1)
x_test.drop(['xcalc_14'], axis = 1)
clf = DecisionTreeClassifier()
y_pred = clf.fit(x_train, y_train).predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy score: " + str(acc))
print("F1 score: " + str(f1_score(y_test, y_pred)))

Accuracy score: 0.9080664468572148
F1 score: 0.058580645161290315


Accuracy score: 0.9080664468572148

F1 score: 0.058580645161290315

In [42]:
# Task 2.2 - binning 2 - equal depth
# chosen feature = xcalc_14
y = df['y']
x = df.drop(['y'], axis=1)
x['xcalc_14_3'] = x['xcalc_14']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, random_state=42)
x_train['xcalc_14_3'] = pd.qcut(x_train['xcalc_14'], 4).cat.codes

In [43]:
# classification:
x_train.drop(['xcalc_14'], axis = 1)
x_test.drop(['xcalc_14'], axis = 1)
clf = DecisionTreeClassifier()
y_pred = clf.fit(x_train, y_train).predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy score: " + str(acc))
print("F1 score: " + str(f1_score(y_test, y_pred)))

Accuracy score: 0.9183316882625953
F1 score: 0.054004086795757515


Accuracy score: 0.9183316882625953

F1 score: 0.054004086795757515

Vyšší přesnost ukázala metida Equal depth, ovšem vyšší f1 skore ukázala metoda Equal width. Pro konečné porovnání bude použita metoda Equal depth.

Na této metodě také ukážu, jak se může přesnost změnit, pokud použiji jiný klasifikátor (v tomto případě kNN):

In [44]:
# porovnani s pouzitim klasifikace pomoci KNN (equal-depth)
clf = KNeighborsClassifier(3)
y_pred = clf.fit(x_train, y_train).predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy score: " + str(acc))
print("F1 score: " + str(f1_score(y_test, y_pred)))

Accuracy score: 0.9597958712224626
F1 score: 0.00931484164769199


Accuracy score: 0.9597958712224626
    
F1 score: 0.00931484164769199

### Edited Nearest Neighbours

In [35]:
# Task 3.1 - other data balancing technique
# Edited Nearest Neighbor
y = df['y']
x = df.drop(['y'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, random_state=42)
enn = EditedNearestNeighbours()
x_resampled_enn, y_resampled_enn = enn.fit_resample(x_train, y_train)

In [36]:
# classification:
clf = DecisionTreeClassifier()
y_pred_enn = clf.fit(x_resampled_enn, y_resampled_enn).predict(x_test)
acc = accuracy_score(y_test, y_pred_enn)
print("Accuracy score: " + str(acc))
print("F1 score: " + str(f1_score(y_test, y_pred_enn)))

Accuracy score: 0.9135224814667031
F1 score: 0.05853033975033152


Accuracy score: 0.9135224814667031

F1 score: 0.05853033975033152

### One-side sampling

In [37]:
# Task 3.2 - other data balancing technique
# One-side sampling
y = df['y']
x = df.drop(['y'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, random_state=42)
os = OneSidedSelection(random_state=42)
x_resampled_os, y_resampled_os = os.fit_resample(x_train, y_train)

In [38]:
# classification:
clf = DecisionTreeClassifier()
y_pred_os = clf.fit(x_resampled_os, y_resampled_os).predict(x_test)
acc = accuracy_score(y_test, y_pred_os)
print("Accuracy score: " + str(acc))
print("F1 score: " + str(f1_score(y_test, y_pred_os)))

Accuracy score: 0.9194069344981834
F1 score: 0.052257235997234024


Accuracy score: 0.9194069344981834
    
F1 score: 0.052257235997234024

One-side sampling vyšel přesněji než ENN, tedy bude použit pro konečné porovnání.

### Classification

In [55]:
# all of the following data are classified using decision tree classifier
# Task 4.1 - classification on unbalanced data
print("Classification on unbalanced data")
y = df['y']
x = df.drop(['y'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, random_state=42)
clf = DecisionTreeClassifier()
y_pred = clf.fit(x_train, y_train).predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy score: " + str(acc))
print("F1 score: " + str(f1_score(y_test, y_pred)))

Classification on unbalanced data

Accuracy score: 0.9186089001827079

F1 score: 0.054823919617598284

In [51]:
# Task 4.2 - classification on balanced data 1 - equal depth binning
y = df['y']
x = df.drop(['y'], axis=1)
x['xcalc_14_3'] = x['xcalc_14']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, random_state=42)
x_train['xcalc_14_3'] = pd.qcut(x_train['xcalc_14'], 4).cat.codes
x_train.drop(['xcalc_14'], axis = 1)
x_test.drop(['xcalc_14'], axis = 1)
clf = DecisionTreeClassifier()
y_pred = clf.fit(x_train, y_train).predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy score: " + str(acc))
print("F1 score: " + str(f1_score(y_test, y_pred)))

Accuracy score: 0.9180460759812672

F1 score: 0.056935717738037704

In [47]:
# Task 4.3 - classification on balanced data 2
# z duvodu dlouheho vypocetniho casu pouziji predchozi vysledky klasifikace metody One-sampling.

### Result analysis

#### Unbalanced data:
Accuracy score: 0.9186089001827079

F1 score: 0.054823919617598284

#### Binning (depth):
Accuracy score: 0.9180460759812672

F1 score: 0.056935717738037704

#### One-side sampling:
Accuracy score: 0.9194069344981834

F1 score: 0.052257235997234024


Nejlépe dle Accuracy score dopadl balancing pomocí one-side sampling, podle F1 score pak binning. Výsledky se při klasifikaci pomocí rozhodovacího stromu příliš neliší. 

Při použití jiných klasifikátorů se výsledky mohou lišit, jak ukázala např. klasifikace při binningu pomocí kNN.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=61c8b527-a4b8-4d15-9e2e-537eeed4064c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>